## Plotting

Being able to plot data is essential. The graphics output option above means we can use any plotting capability that will give us `html` or `svg` so if you have a favourite charing library give it a go.

[plotly.js](https://plot.ly/javascript/) is a fully featued charting library and an officially supported wrapper ijavascript alreeady exists. [js-notebook-plotly](https://github.com/plotly/plotly-notebook-js) its in beta but we're using it here.

Here are some simple plots and warm up exercises.

In [1]:
var fs = require("fs");
var Plot = require('plotly-notebook-js');
var parse = require('csv-parse/lib/sync');

'use strict'

#### Example - muliple 1D curves / timeseries / vectors




In [2]:
import range from 'lodash/range';

var x = range(0,100).map(n => 4*Math.PI*n/100);

var trace1 = { 
    x,
    y: x.map(x => Math.cos(x)),
    type: 'bar',
    name: 'cosine'
};

var trace2 = { 
    x,
    y: x.map(x => Math.sin(x)),
    type: 'lines',
    name: 'sine'
};

var myPlot = Plot.createPlot([trace1, trace2], { title: 'plot multiple curves' });

$$html$$ = myPlot.render();

#### Example - scatter & bubble plotting

In [11]:
var groupBy = require('lodash/groupBy');
var uniqBy = require('lodash/uniqBy');

var raw_csv_string = fs.readFileSync("datasets/wine.data.csv").toString();

var CLASS_COL = 13;

var dataset = parse(raw_csv_string, {auto_parse: true});
var classes = uniqBy(dataset.map(d => d[CLASS_COL]), d => d);
console.log(classes)

[ 1, 2, 3 ]


#### 2D Scatter Plotting

A 2d scatter plot with a seperate `trace` per class label

In [4]:
var traces = classes.map(label => {
    var data = dataset.filter(d => d[CLASS_COL] === label);
    return {
        x: data.map(d => d[0]).map(f => parseFloat(f)),
        y: data.map(d => d[6]).map(f => parseFloat(f)),
        mode: 'markers',
        type: 'scatter',
        name: `Class ${label}`
    };
});

var layout = {
    title: 'Wines',
    xaxis: { title: 'Alcohol Content ABV'},
    yaxis: { title: 'Flavinoids'},
    height: 700
};

$$html$$ = Plot.createPlot(traces, layout).render()

#### 3D Scatter Plotting

In [5]:
var traces = classes.map(label => {
    var data = dataset.filter(d => d[CLASS_COL] === label);
    return {
        x: data.map(d => d[0]).map(f => parseFloat(f)),
        y: data.map(d => d[5]).map(f => parseFloat(f)),
        z: data.map(d => d[6]).map(f => parseFloat(f)),        
        mode: 'markers',
        type: 'scatter3d',
        name: `Class ${label}`,
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5},
                opacity: 0.8
            },
        };
    });

var layout = {
    width: 1000,
    height: 1000,
    title: 'Wines',
    xaxis: { title: 'Alcohol Content ABV'},
    yaxis: { title: 'Flavinoids'},
    zaxis: { title: 'Total Phenols'},
};

$$html$$ = Plot.createPlot(traces, layout).render()

#### Bubble Charts

In [6]:
var traces = classes.map(label => {
    var data = dataset.filter(d => d[CLASS_COL] === label);
    return {
        x: data.map(d => d[0]).map(f => parseFloat(f)),
        y: data.map(d => d[6]).map(f => parseFloat(f)),
        mode: 'markers',
        marker: {
            size: data.map(d => d[10]).map(f => parseFloat(f)),
            sizeref: 0.2
        },
        type: 'scatter',
        name: `Class ${label}`
    };
});

var layout = {
    title: 'Wines (Bubble Size = Color Intensity)',
    xaxis: { title: 'Alcohol Content ABV'},
    yaxis: { title: 'Flavinoids'},
    height: 700
};

$$html$$ = Plot.createPlot(traces, layout).render()

In [7]:
var classColors = ['rgb(93, 164, 214)', 'rgb(255, 144, 14)',  'rgb(44, 160, 101)', 'rgb(255, 65, 54)'];

var singleTrace = {
    x: dataset.map(d => d[0]).map(f => parseFloat(f)),
    y: dataset.map(d => d[6]).map(f => parseFloat(f)),
    mode: 'markers',
    marker: {
        color: dataset.map(d => d[CLASS_COL]).map(f => classColors[parseInt(f)]),
        size: dataset.map(d => d[9]).map(f => parseFloat(f)),
        sizeref: 0.2
    },
    type: 'scatter'
};

var layout = {
    title: 'Wines (Bubble Size = Color Intensity)',
    xaxis: { title: 'Alcohol Content ABV'},
    yaxis: { title: 'Flavinoids'},
    height: 700
};

$$html$$ = Plot.createPlot([singleTrace], layout).render()

### 2D Heatmaps

In [8]:
var X = [];
var step = 0.01;

// build a 2d array of coordinates
for (var i = -1; i <= 1; i += step) {
    var x = [];
    for (var j = -1; j <= 1; j += step) {
        x.push([i, j])
    }
    X.push(x)
}

// compute the "heat" data
var H = X.map(col => {
    return col.map(x => {
        const r = x.reduce((a,xi) => a + xi*xi, 0);
        var F = 3;
        var A = Math.cos(Math.PI*r/4);
        return A*Math.cos(2*Math.PI*F*r);
    })
});

var data = {
    z: H,
    type: "heatmap"
}

var layout = {
    title: "damped circular wavefront",
    width: 700,
    height: 700
}

$$html$$ = Plot.createPlot([data], layout).render()

### 2D Surface Plots

Note this relies on the 2D Heatmap example from the cell above, make sure its been run

In [9]:
$$html$$ = Plot.createPlot([{ // data
    z: H,
    type: "surface"
}], { // layout
    title: "damped circular wavefront",
    width: 700,
    height: 700
}).render()

In [10]:
import uniq from 'lodash/uniq';

function unpack(data, key) { return data.map(row => row[key]) } 

var raw_csv_string = fs.readFileSync("datasets/who_life_expectancy.csv").toString();

var dataset = parse(raw_csv_string);
var data = dataset.slice(1);
var regions = uniq(data.map(r => r[0]))
console.log(regions)

var traces = regions.map( region => ({
    x: data.filter(d => d[0] === region).map(r => r[1]).map(x => parseInt(x)),
    y: data.filter(d => d[0] === region).map(r => r[2]).map(parseFloat),
    mode: 'markers',
    type: 'scatter',
    name: region
}));

$$html$$ = Plot.createPlot(traces, {title: 'Life expectancy'}).render()

[ 'Africa',
  'Americas',
  'South-East Asia',
  'Europe',
  'Eastern Mediterranean',
  'Western Pacific',
  'Global' ]
